# Amazon SageMaker와 함께 채팅 완료 클라이언트를 사용하는 방법

EasyLLM은 `gpt-3.5-turbo` 및 `gpt-4`를 오픈 소스 모델로 대체하는 추상화 계층으로 사용할 수 있습니다.

클라이언트만 변경하여 OpenAI API에서 자체 애플리케이션을 변경할 수 있습니다.

채팅 모델은 일련의 메시지를 입력으로 사용하고 AI가 작성한 메시지를 출력으로 반환합니다.

이 가이드에서는 몇 가지 예제 API 호출을 통해 채팅 형식을 보여줍니다.

## 0. 설정

Amazon SageMaker에서 `easyllm`을 사용하려면 먼저 모델을 SageMaker 엔드포인트에 배포해야 합니다. 아래 블로그 게시물 중 하나를 따라 이 작업을 수행할 수 있습니다.

* [Amazon SageMaker에 Llama 2 7B/13B/70B 배포](https://www.philschmid.de/sagemaker-llama-llm)
* [Amazon SageMaker에 Falcon 7B & 40B 배포](https://www.philschmid.de/sagemaker-falcon-llm)
* [Amazon SageMaker용 Hugging Face LLM 추론 컨테이너 소개](https://www.philschmid.de/sagemaker-huggingface-llm)

엔드포인트 배포가 완료되면 엔드포인트 이름을 복사합니다. 엔드포인트 이름이 `model` 매개변수가 됩니다. AWS 관리 콘솔의 Amazon SageMaker에서 "추론" -> "엔드포인트" -> "이름"으로 이동하거나 sagemaker SDK를 사용하여 모델을 배포한 경우 `predictor.endpoint_name` 속성에서 엔드포인트 이름을 가져올 수 있습니다.


### 1. easyllm 라이브러리 가져오기

In [ ]:
# 필요한 경우 EasyLLM Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm 

In [6]:
# EasyLLM API를 호출하기 위한 EasyLLM Python 라이브러리를 가져옵니다.
import easyllm

### 2. 채팅 API 호출 예제

채팅 API 호출에는 두 가지 필수 입력이 있습니다.
- `model`: 사용하려는 모델의 이름(예: `huggingface-pytorch-tgi-inference-2023-08-08-14-15-52-703`) 또는 API만 호출하려면 비워 둡니다.
- `messages`: 메시지 개체 목록이며 각 개체에는 두 가지 필수 필드가 있습니다.
    - `role`: 메신저의 역할( `system`, `user` 또는 `assistant`)
    - `content`: 메시지 내용(예: `아름다운 시를 써주세요`)

OpenAI API와 비교하여 `huggingface` 모듈은 프롬프트 및 중지 시퀀스를 사용자 지정하는 데 사용할 수 있는 `prompt_builder` 및 `stop_sequences` 매개변수도 노출합니다. EasyLLM 패키지에는 프롬프트 빌더 유틸리티가 함께 제공됩니다.

채팅 형식이 실제로 어떻게 작동하는지 확인하기 위해 예제 채팅 API 호출을 살펴보겠습니다.

In [6]:
import os 
# 프롬프트 빌더용 환경 변수 설정
os.environ["HUGGINGFACE_PROMPT"] = "llama2" # vicuna, wizardlm, stablebeluga, open_assistant
os.environ["AWS_REGION"] = "us-east-1"  # 해당 지역으로 변경
# os.environ["AWS_ACCESS_KEY_ID"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요
# os.environ["AWS_SECRET_ACCESS_KEY"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요

from easyllm.clients import sagemaker

# 환경 변수를 사용하지 않고 구성 변경
# sagemaker.prompt_builder = "llama2"
# sagemaker.api_aws_access_key="xxx"
# sagemaker.api_aws_secret_key="xxx"

# SageMaker 엔드포인트 이름
MODEL="huggingface-pytorch-tgi-inference-2023-08-08-14-15-52-703"

response = sagemaker.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "\n당신은 도움이 되고 정중하며 정직한 조수입니다. 항상 안전하면서 가능한 한 도움이 되도록 답변하십시오. 답변에는 유해하거나 비윤리적이거나 인종 차별적이거나 성 차별적이거나 유독하거나 위험하거나 불법적인 내용이 포함되어서는 안 됩니다. 답변이 사회적으로 편향되지 않고 긍정적인지 확인하십시오.\n\n질문이 이해가 되지 않거나 사실적으로 일관성이 없는 경우 올바르지 않은 내용을 답변하는 대신 이유를 설명하십시오. 질문에 대한 답변을 모르는 경우 거짓 정보를 공유하지 마십시오."},
        {"role": "user", "content": "Amazon SageMaker에 대해 알려주시겠어요?"},
    ],
      temperature=0.9,
      top_p=0.6,
      max_tokens=1024,
      debug=False,
)
response



{'id': 'hf-2qYJ06mvpP',
 'object': 'chat.completion',
 'created': 1691507348,
 'model': 'huggingface-pytorch-tgi-inference-2023-08-08-14-15-52-703',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': " 물론입니다! Amazon SageMaker는 Amazon Web Services(AWS)에서 제공하는 클라우드 기반 기계 학습 플랫폼입니다. 데이터 과학자와 기계 학습 실무자가 기계 학습 모델을 더 쉽고 효율적으로 구축, 학습 및 배포할 수 있도록 지원합니다. SageMaker를 사용하면 데이터 준비, 모델 학습 및 모델 배포를 포함한 광범위한 기계 학습 작업을 단일 플랫폼 내에서 모두 수행할 수 있습니다.\nAmazon SageMaker의 주요 기능 중 일부는 다음과 같습니다.\n1. 데이터 랭글링: SageMaker는 데이터 정리, 특성 공학 및 데이터 변환을 포함한 데이터 준비를 위한 다양한 도구를 제공합니다.\n2. 학습 및 하이퍼파라미터 튜닝: 사용자는 SageMaker의 기본 제공 알고리즘 또는 자체 사용자 지정 알고리즘을 사용하여 기계 학습 모델을 학습할 수 있습니다. 이 플랫폼은 또한 모델 성능을 향상시키는 데 도움이 될 수 있는 자동 하이퍼파라미터 튜닝을 제공합니다.\n3. 모델 배포: 모델이 학습되고 최적화되면 SageMaker를 통해 사용자는 Amazon S3 및 Amazon EC2와 같은 AWS 서비스는 물론 온프레미스 환경을 포함한 다양한 환경에 모델을 배포할 수 있습니다.\n4. 협업 및 관리: SageMaker는 버전 관리, 재현성 및 팀 기반 워크플로를 포함한 협업 및 모델 관리를 위한 도구를 제공합니다.\n5. 다른 AWS 서비스와의 통합: SageMaker는 Amazon S3, Amazon Redshift 및 Amazon EMR

보시다시피 응답 개체에는 몇 가지 필드가 있습니다.
- `id`: 요청 ID
- `object`: 반환된 개체 유형(예: `chat.completion`)
- `created`: 요청 타임스탬프
- `model`: 응답을 생성하는 데 사용된 모델의 전체 이름
- `usage`: 프롬프트, 완성 및 합계를 계산하여 회신을 생성하는 데 사용된 토큰 수
- `choices`: 완성 개체 목록(`n`을 1보다 크게 설정하지 않은 경우 하나만)
    - `message`: 모델에서 생성한 메시지 개체( `role` 및 `content` 포함)
    - `finish_reason`: 모델이 텍스트 생성을 중지한 이유(`stop` 또는 `max_tokens` 제한에 도달한 경우 `length`)
    - `index`: 선택 항목 목록에서 완성의 인덱스

다음을 사용하여 회신만 추출합니다.

In [7]:
print(response['choices'][0]['message']['content'])

 물론입니다! Amazon SageMaker는 Amazon Web Services(AWS)에서 제공하는 클라우드 기반 기계 학습 플랫폼입니다. 데이터 과학자와 기계 학습 실무자가 기계 학습 모델을 더 쉽고 효율적으로 구축, 학습 및 배포할 수 있도록 지원합니다. SageMaker를 사용하면 데이터 준비, 모델 학습 및 모델 배포를 포함한 광범위한 기계 학습 작업을 단일 플랫폼 내에서 모두 수행할 수 있습니다.
Amazon SageMaker의 주요 기능 중 일부는 다음과 같습니다.
1. 데이터 랭글링: SageMaker는 데이터 정리, 특성 공학 및 데이터 변환을 포함한 데이터 준비를 위한 다양한 도구를 제공합니다.
2. 학습 및 하이퍼파라미터 튜닝: 사용자는 SageMaker의 기본 제공 알고리즘 또는 자체 사용자 지정 알고리즘을 사용하여 기계 학습 모델을 학습할 수 있습니다. 이 플랫폼은 또한 모델 성능을 향상시키는 데 도움이 될 수 있는 자동 하이퍼파라미터 튜닝을 제공합니다.
3. 모델 배포: 모델이 학습되고 최적화되면 SageMaker를 통해 사용자는 Amazon S3 및 Amazon EC2와 같은 AWS 서비스는 물론 온프레미스 환경을 포함한 다양한 환경에 모델을 배포할 수 있습니다.
4. 협업 및 관리: SageMaker는 버전 관리, 재현성 및 팀 기반 워크플로를 포함한 협업 및 모델 관리를 위한 도구를 제공합니다.
5. 다른 AWS 서비스와의 통합: SageMaker는 Amazon S3, Amazon Redshift 및 Amazon EMR과 같은 다른 AWS 서비스와 통합되어 포괄적인 기계 학습 플랫폼을 제공합니다.
전반적으로 Amazon SageMaker는 모든 규모의 조직이 기계 학습 모델을 보다 효율적이고 효과적으로 구축하고 배포하는 데 도움이 되는 강력한 플랫폼입니다.


대화 기반이 아닌 작업도 첫 번째 사용자 메시지에 지침을 배치하여 채팅 형식에 맞출 수 있습니다.

예를 들어 모델에게 해적 블랙비어드 스타일로 비동기 프로그래밍을 설명하도록 요청하려면 다음과 같이 대화를 구성할 수 있습니다.

In [3]:
# 시스템 메시지가 있는 예제
response = sagemaker.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "당신은 도움이 되는 조수입니다."},
        {"role": "user", "content": "수학 교사 스타일로 비동기 프로그래밍을 설명해주세요."},
    ],
)

print(response['choices'][0]['message']['content'])


 아, 나의 사랑하는 제자여, 비동기 프로그래밍을 가장 즐겁고 직관적인 방식으로 설명해 주겠네! *안경을 고쳐 쓴다*

비동기 프로그래밍은, 알다시피, 복잡한 방정식을 푸는 것과 같네. *칠판에 쓴다* 즉각적인 주의가 필요한 문제가 있지만, 그냥 앉아서 해결책이 나타나기를 기다릴 수만은 없네. *엄지손가락을 만지작거리는 사람 흉내* 아니, 아니, 나의 젊은 제자여! 창의력과 독창성의 힘을 사용하여 병렬로 문제를 해결할 방법을 찾아야 하네! *윙크*
이제 수학에서는 방정식을 풀기 위해 치환, 소거 또는 때로는 복소수의 기교와 같은 기술을 자주 사용하네. *끄덕인다* 하지만 비동기 프로그래밍에서는 "비동기 작업"이라는 것을 사용하여 단순한 "기다렸다가 보자" 접근 방식 이상의 것이 필요한 문제를 해결하네. *씨익 웃는다*
이렇게 생각해보게. 해야 할 작업이 많지만 동시에 모두 수행할 수는 없다고 상상해보게. 데이터베이스에서 일부 데이터를 가져와 처리한 다음 일부 계산을 수행해야 할 수도 있네. *키보드로 입력하는 흉내* 하지만 잠깐! 각 작업이 끝날 때까지 그냥 앉아서 기다릴 수만은 없네, 그렇지 않으면 몇 시간 동안 엄지손가락만 만지작거리고 있을 걸세! *웃는다*
그래서 어떻게 해야 할까? *씨익 웃는다* 각 작업을 더 작고 관리하기 쉬운 조각으로 나누고 각 조각에 특별한 "비동기 모자"를 씌우는 걸세! *윙크* 이 모자를 사용하면 각 조각이 다른 조각이 끝날 때까지 기다리지 않고 독립적으로 작업을 수행할 수 있네. *끄덕인다*
예를 들어, 한 조각에는 데이터베이스에서 데이터를 가져오는 작업을, 다른 조각에는 처리하는 작업을, 또 다른 조각에는 계산을 수행하는 작업을 할당할 수 있네. *모자를 나눠주는 흉내* 그런 다음 그냥 앉아서 각 조각이 다른 조각이 끝날 때까지 기다리지 않고 작업을 수행하는 것을 지켜볼 수 있네! *웃는다*
하지만 잠깐, 더 있네! *흥분해서* 비동기 프로그래밍을 사용하면 "콜백"이라는 것을 사용하여 모든 것이 올바른 순서대로 완료되도록 

In [4]:
# 시스템 메시지가 없고 디버그 플래그가 켜진 예제:
response = sagemaker.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "해적 블랙비어드 스타일로 비동기 프로그래밍을 설명해주세요."},
    ]
)

print(response['choices'][0]['message']['content'])


 이런 젠장! 이 육지 놈들이 비동기 프로그래밍이라는 것에 대해 알고 싶어 하는 건가? 그럼 위대한 검은 수염 자신의 이야기를 잘 들어보게, 내가 어떻게 작동하는지 이야기해 줄 테니!

아호이, 친구들! 비동기 프로그래밍은 넓은 바다를 항해하는 것과 같네. 알다시피, 배가 있어야 하고, 그 배는 "스레드"라고 불리네. 이제 "검은 수염, 배를 조종할 수 없다면 배가 무슨 소용이 있나?"라고 생각할지도 모르겠네. 그리고 거기에 대해 나는 "아르르, 육지 놈들은 너무 작게 생각하고 있군!"이라고 말하겠네. 보게, 비동기 프로그래밍을 사용하면 여러 "스레드"가 동시에 바다를 항해하면서 각자 자기 일을 할 수 있네. 그리고 그건 정말 강력한 것이라네, 친구들!
하지만 잠깐, 더 있네! 알다시피, 이 스레드들은 각자 선원과 임무를 가진 다른 배와 같네. 그리고 그들은 서로 다른 속도로 바다를 항해하고 있네! 어떤 스레드는 빠르게 항해하고, 다른 스레드는 느리게 항해할 수도 있네. 그리고 그게 바로 그 아름다움이라네, 친구들! 중요한 일로 바쁜 스레드가 있는 동안 다른 스레드는 편안하게 낮잠을 자고 있을 수도 있네. 마치 자기 마음대로 움직이는 함대를 마음대로 부리는 것과 같네!
이제 무슨 생각을 하는지 알겠네. "검은 수염, 이 모든 배들이 서로 충돌하지 않도록 어떻게 하나?" 그리고 거기에 대해 나는 "아르르, 그게 바로 비동기 프로그래밍의 마법이라네, 친구들!"이라고 말하겠네. 알다시피, 각 스레드는 자기 항로를 따라가고, 메시지를 통해 서로 통신하네. 마치 넓은 바다에서 다른 배에 메시지를 보내는 것과 같지만, 메시지 대신 특별한 종류의 코드를 사용하네. 그리고 그 코드는 각 스레드가 어디로 가야 하고 무엇을 해야 하는지 보여주는 지도와 같네.
하지만 잠깐, 더 있네! 알다시피, 이 스레드들은 배의 다른 선원들과 같네. 어떤 선원은 칼솜씨가 뛰어나고, 다른 선원은 항해술이 뛰어나네. 그리고 그들은 함께 일하면서 각자 자기 역할을 하여 배가 순조롭게 항해하도록 하네

### 3. 소수 샷 프롬프트

경우에 따라 원하는 것을 모델에게 말하는 것보다 보여주는 것이 더 쉽습니다.

원하는 것을 모델에게 보여주는 한 가지 방법은 가짜 예제 메시지를 사용하는 것입니다.

예를 들어:

In [5]:
# 비즈니스 전문 용어를 더 간단한 말로 번역하도록 모델을 준비시키는 가짜 소수 샷 대화 예제
response = sagemaker.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "당신은 도움이 되고 패턴을 따르는 조수입니다."},
        {"role": "user", "content": "다음 기업 전문 용어를 평이한 영어로 번역하는 것을 도와주세요."},
        {"role": "assistant", "content": "물론입니다, 기꺼이 도와드리겠습니다!"},
        {"role": "user", "content": "새로운 시너지가 최고 수익 성장을 이끌 것입니다."},
        {"role": "assistant", "content": "함께 잘 작동하는 것들이 수익을 증가시킬 것입니다."},
        {"role": "user", "content": "증가된 레버리지 기회에 대해 논의할 시간이 더 있을 때 다시 논의합시다."},
        {"role": "assistant", "content": "더 잘하는 방법에 대해 덜 바쁠 때 나중에 이야기합시다."},
        {"role": "user", "content": "이 늦은 전환은 고객 결과물에 대해 바다를 끓일 시간이 없다는 것을 의미합니다."},
    ],
)

print(response["choices"][0]["message"]["content"])


"원래 고객을 위해 계획했던 모든 것을 할 시간이 없으므로 가장 중요한 일에 집중하고 나중에 '바다를 끓여야' 합니다."


대화 엔지니어링에 대한 모든 시도가 처음부터 성공하는 것은 아닙니다.

첫 번째 시도가 실패하면 모델을 준비하거나 조정하는 다양한 방법을 실험하는 것을 두려워하지 마십시오.

예를 들어, 한 개발자는 모델이 더 높은 품질의 응답을 제공하도록 조정하는 데 도움이 되도록 "지금까지 훌륭했습니다. 완벽했습니다."라는 사용자 메시지를 삽입했을 때 정확도가 증가하는 것을 발견했습니다.

모델의 신뢰성을 높이는 방법에 대한 더 많은 아이디어는 [신뢰성 향상 기술](../techniques_to_improve_reliability.md)에 대한 가이드를 참조하십시오. 비채팅 모델용으로 작성되었지만 많은 원칙이 여전히 적용됩니다.